In [12]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [13]:
st = tf.SparseTensor([[0, 0]], [1], [2,2])
print(sess.run(st))

SparseTensorValue(indices=array([[0, 0]]), values=array([1], dtype=int32), dense_shape=array([2, 2]))


In [17]:
dense = tf.sparse_tensor_to_dense(st)
print(sess.run(dense))

[[1 0]
 [0 0]]


In [22]:
batch_y = tf.random_normal(shape=(10, 5))
sess.run(batch_y).shape

(10, 5)

In [25]:
a = tf.one_hot(tf.constant([0, 1, 2]), depth=3)
sess.run(a)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

# choose rows from tensor

In [50]:
inds = [0,1,2]

In [51]:
def sparse_slice(y, inds):
    st = tf.SparseTensor(
        indices=list(map(lambda x : (x[1], x[0]), enumerate(inds))), 
        values=np.ones(len(inds), dtype=np.float32), 
        dense_shape=(len(inds), y.shape[1])
    )
    return tf.transpose(tf.sparse_tensor_dense_matmul(st, tf.transpose(y)))

In [53]:
sess.run(sparse_slice(batch_y, inds)).shape

(10, 3)

# group loss

In [54]:
from collections import namedtuple

In [55]:
group_t = namedtuple('group_t', ['id', 'inds'])

In [61]:
groups = [group_t(2, [0, 1, 2])]

In [78]:
def to_lower_labels(labels, groups):
    a = np.identity(labels.shape[1])
    for group in groups:
        r = np.zeros(labels.shape[1])
        r[group.inds] = 1.0 / len(group.inds)
        a[group.id] = r
    A = tf.constant(a, dtype=tf.float32)
    return tf.matmul(labels, A)
    
def to_upper_labels(labels, groups):
    # find inds to use
    used = set(sum(map(lambda x : x.inds, groups), []))
    notused = set(range(labels.shape[1])) - used
    a = np.zeros((labels.shape[1], labels.shape[1] - len(used) + len(groups)))
    for i, ind in enumerate(sorted(notused)):
        a[i] = 
    
    
def GroupLoss(labels, logits, groups):
    lower_labels = to_lower_labels(labels)
    upper_labels = to_upper_labels(labels)
        

In [76]:
labels = tf.constant([[0, 0, 1], [0, 1, 0], [0, 0, 1]], dtype=tf.float32)

In [77]:
print(sess.run(labels))
print(sess.run(to_lower_labels(labels, groups)))

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[0.33333334 0.33333334 0.33333334]
 [0.         1.         0.        ]
 [0.33333334 0.33333334 0.33333334]]


In [80]:
to_upper_labels(labels, groups)

[0, 1, 2]

In [81]:
set({1,2,3}) - set({2,1})

{3}

In [83]:
np.eye(3, 2)

array([[1., 0.],
       [0., 1.],
       [0., 0.]])

# sparse quadratic form

In [122]:
x = tf.SparseTensor(sorted([[0, 0], [4, 1], [1,1]]), tf.cast([3, 1,1], dtype=tf.float32), [5, 2])

In [123]:
sess.run(tf.sparse_tensor_to_dense(x))

array([[3., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [124]:
C = tf.constant(np.ones((2, 2)), dtype=tf.float32)

In [125]:
sess.run(C)

array([[1., 1.],
       [1., 1.]], dtype=float32)

In [126]:
y = tf.sparse_tensor_dense_matmul(x, C)

In [128]:
sess.run(y).shape

(5, 2)

In [130]:
z = tf.transpose(tf.sparse_tensor_dense_matmul(x, tf.transpose(y)))

In [131]:
sess.run(z)

array([[9., 3., 0., 0., 3.],
       [3., 1., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [3., 1., 0., 0., 1.]], dtype=float32)

### standart mul with dense

In [135]:
sess.run(tf.sparse_tensor_to_dense(x * y))

array([[9., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [137]:
q = tf.sparse_reduce_sum(tf.sparse_tensor_dense_matmul(x, C) * x, axis=1)

In [138]:
sess.run(q)

array([9., 1., 0., 0., 1.], dtype=float32)

# sparse slice and split

In [140]:
sess.run(tf.sparse_tensor_to_dense(x))

array([[3., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [143]:
a, b = tf.sparse_split(sp_input=x, num_split=2, axis=1)

In [144]:
sess.run(tf.sparse_tensor_to_dense(a))

array([[3.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [145]:
sess.run(tf.sparse_tensor_to_dense(b))

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.]], dtype=float32)

### many dims

In [150]:
st = tf.SparseTensor(sorted([[0, 0], [1, 0], [1, 6], [0, 8]]), tf.cast([1, 2, 3, 4], tf.float32), [2,10])

In [151]:
sess.run(tf.sparse_tensor_to_dense(st))

array([[1., 0., 0., 0., 0., 0., 0., 0., 2., 0.],
       [3., 0., 0., 0., 0., 0., 4., 0., 0., 0.]], dtype=float32)

In [152]:
# 2 3 5

In [158]:
a,b,c = tf.sparse_slice(st, [0, 0], [2, 2]), tf.sparse_slice(st, [0, 2], [2, 3]), tf.sparse_slice(st, [0, 5], [2,5])

In [159]:
sess.run(tf.sparse_tensor_to_dense(a))

array([[1., 0.],
       [3., 0.]], dtype=float32)

In [160]:
sess.run(tf.sparse_tensor_to_dense(b))

array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

In [161]:
sess.run(tf.sparse_tensor_to_dense(c))

array([[0., 0., 0., 2., 0.],
       [0., 4., 0., 0., 0.]], dtype=float32)

### tensor as split

In [164]:
a,b,c = tf.sparse_slice(st, [0, 0], [st.get_shape()[0], 2]), tf.sparse_slice(st, [0, 2], [st.get_shape()[0], 3]), tf.sparse_slice(st, [0, 5], [st.get_shape()[0],5])

In [165]:
sess.run(tf.sparse_tensor_to_dense(a))

array([[1., 0.],
       [3., 0.]], dtype=float32)